In [93]:
# from .utils import *
# from .timedistance import *
# from .delay import *
# from .route import *
from math import sin, cos, sqrt, atan2, radians, inf
from queue import PriorityQueue
from math import inf
import networkx as nx

from utils import *
from algo import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# neighbours()

In [ ]:
def stochastic_dijkstra(self, 
                            start: Union[Name,ID],     # start node (either name or id. If id, names_format to False)
                            end: Union[Name,ID],       # end node (either name or id. If id, names_format to False)
                            arr_time:Union[str,Time],  # target arrival time (ex '12:45:00' or timestamp)
                            threshold = 0.8,           # target probability of making all connections in time
                            names_format=True          # If we want to use IDs for start and end set to False
                           ) -> Optional[Route]:    

    prev = {}                                                                   # predecessor of current node on shortest path 
    distances = {v: TimeDistance(time=arr_time) for v in nx.nodes(self.G)}      # initialize distances from end to any given node
    visited = set()                                                             # nodes we've visited
    # prioritize nodes from start -> node with the shortest distance!
    queue = PriorityQueue()                        
    distances[end].updated()                                                    # dist from end -> end is zero
    queue.put((distances[end], end))                                               # add end node for exploration
    # main search phase of algorithm
    while not queue.empty():
        _, curr = queue.get()
        visited.add(curr)
        for _,neighbor,properties in neighbours(curr):              # look at curr's adjacent nodes
            new_dist = distances[curr] + properties                 # if we found a shorter path 
            if new_dist < distances[neighbor]:                      # update the distance, we found a shorter one!
                distances[neighbor] = new_dist.updated()            # update the previous node to be prev on new shortest path
                prev[neighbor] = curr                               # if we haven't visited the neighbor
                if neighbor not in visited:                         # insert into priority queue and mark as visited
                    visited.add(neighbor)
                    queue.put((distances[neighbor],neighbor))          # prioritize search on small distances
    # compute the route 
    return build_route()

In [ ]:
def build_route() -> Optional[Route]:
    """
    Get the shortest path of nodes by going backwards through prev list
    """
    if start not in prev.keys():
        return None
    
    route = Route(self.G)
    node = start
    dist = distances[start]
    path = []
    while node != end:
        route.append(node,dist)
        node = prev[node]
        dist = distances[node]
    route.append(node,dist) 
    return route

In [94]:
def build_route(prev_trip_id, prev, distances, conn_datas, start_id, end_id):

    if start_id not in prev.keys():
        return None
    
    node = start_id
    # prev_id = prev[start_id]
    trip_id = prev_trip_id[node]
    dist = distances[start_id]
    path = []
    path_conn_datas = []
    while node != end_id:
        path.append((trip_id, node, dist))
        path_conn_datas.append(conn_datas[node])

        node = prev[node]
        trip_id = prev_trip_id[node]
        dist = distances[node]
    # route.append(node,dist)
    path.append((trip_id, node, dist))
    path_conn_datas.append(conn_datas[node])
    return path, pd.concat(path_conn_datas, axis=1).T

In [97]:
end_id = ZURICH_HB_ID
end_time = Time(h=10).in_seconds()
start_id = 8591192


distances = {}      # stores travel_times
prev = {}           # stores predecessor
prev_trip_id = {}   # stores prev_trip_id
# prev_travel_times = {}
conn_datas = {}     # TODO we can replace this with only needed data

visited = set()     # stores already visited stop_ids
queue = PriorityQueue()

distances[end_id] = 0
prev[end_id] = None
prev_trip_id[end_id] = None
conn_datas[end_id] = None
# prev_travel_times[end_id] = 0

queue.put((distances[end_id], (end_id, end_time)))

# TODO? Add thing to stop once we reach target?
while not queue.empty():
    _, (curr_id, curr_time) = queue.get()
    if curr_id == start_id:
        break

    visited.add(curr_id)
    for trip_id, neighbor_id, neighbor_dep_time_s, conn_data in neighbors(curr_id, curr_time, prev_trip_id[curr_id]):
        new_dist = end_time - neighbor_dep_time_s
        # new_dist = distances[curr_id] + neighbor_weight
        if new_dist < distances.get(neighbor_id, inf):

            distances[neighbor_id] = new_dist
            prev[neighbor_id] = curr_id
            prev_trip_id[neighbor_id] = trip_id
            conn_datas[neighbor_id] = conn_data

            if neighbor_id not in visited:
                visited.add(neighbor_id)
                # prev_dep_time
                queue.put((distances[neighbor_id], (neighbor_id, neighbor_dep_time_s)))



In [98]:
path, path_conn_datas=build_route(prev_trip_id, prev, distances, conn_datas, start_id, end_id)
path_conn_datas

,trip_id,dep_stop_id,arr_stop_id,mean,std,dep_time_s,arr_time_s,weight
0,692.TA.26-33-B-j19-1.2.H,8591192,8591376,0,66,34200.0,34260.0,60
53109,692.TA.26-33-B-j19-1.2.H,8591376,8591411,0,66,34260,34320,224
9,walk,8591411,8591373,0,0,NaN,NaN,616
52849,450.TA.26-7-B-j19-1.1.H,8591373,8591174,0,66,35100,35160,93.0
16,walk,8591174,8503088,0,0,NaN,NaN,675.0
0,82.TA.26-4-A-j19-1.4.R,8503088,8503090,0,66,35880.0,35940.0,120


In [88]:


# path_conn_datas[0]
df_path_conns = pd.concat(path_conn_datas, axis=1).T
# df_path_conns.sort_values(by='arr_time_s', ascending=True, inplace=True)
df_path_conns

,trip_id,dep_stop_id,arr_stop_id,mean,std,dep_time_s,arr_time_s,weight
0,692.TA.26-33-B-j19-1.2.H,8591192,8591376,0,66,34200.0,34260.0,60
53109,692.TA.26-33-B-j19-1.2.H,8591376,8591411,0,66,34260,34320,224
9,walk,8591411,8591373,0,0,NaN,NaN,616
52849,450.TA.26-7-B-j19-1.1.H,8591373,8591174,0,66,35100,35160,93.0
16,walk,8591174,8503088,0,0,NaN,NaN,675.0
0,82.TA.26-4-A-j19-1.4.R,8503088,8503090,0,66,35880.0,35940.0,120


In [68]:

def build_route_df(route, end_time):
    df_route = pd.DataFrame(route)
    df_route.rename({0:'trip_id', 1:'dep_stop_id', 2:'dep_time'}, axis=1, inplace=True)
    # df_route['dep_time'] = end_time - df_route['dep_time']
    df_route.sort_values(by='dep_time', ascending=True, inplace=True)
    return df_route

build_route_df(route, end_time)

,trip_id,dep_stop_id,dep_time
6,None,8503090,0.0
5,82.TA.26-4-A-j19-1.4.R,8503088,120.0
4,walk,8591174,795.0
3,450.TA.26-7-B-j19-1.1.H,8591373,900.0
2,walk,8591411,1516.0
1,692.TA.26-33-B-j19-1.2.H,8591376,1740.0
0,692.TA.26-33-B-j19-1.2.H,8591192,1800.0
